In [1]:
from langchain_ollama import ChatOllama

In [2]:
llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0,
        streaming=True
    )

In [3]:
llm.invoke("hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-02-25T14:06:24.71600414Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4460336512, 'load_duration': 4082975157, 'prompt_eval_count': 26, 'prompt_eval_duration': 269000000, 'eval_count': 8, 'eval_duration': 105000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-0f870d45-129d-46ef-a6a7-e377628e6244-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

In [4]:
from langchain_ollama import OllamaEmbeddings

In [5]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

In [6]:
embeddings

OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434', client_kwargs={})

In [7]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
file = "/home/namnh1/rag-llm-chatbot/documents/Bản sao của SRS template editing.pdf"
chunk_size = 700
chunk_overlap = 200
loader = PyMuPDFLoader(file)
documents = loader.load()

In [9]:
splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", "(?<=\. )", " ", ""]
        )

In [10]:
docs = splitter.split_documents(documents)

In [11]:
docs[0]

Document(metadata={'source': '/home/namnh1/rag-llm-chatbot/documents/Bản sao của SRS template editing.pdf', 'file_path': '/home/namnh1/rag-llm-chatbot/documents/Bản sao của SRS template editing.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.4', 'title': 'Bản sao của SRS template editing', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Skia/PDF m134 Google Docs Renderer', 'creationDate': '', 'modDate': '', 'trapped': ''}, page_content='TÀI LIỆU PHÂN TÍCH NGHIỆP VỤ HỆ \nTHỐNG GIÁM SÁT HÀNH VI EVISION \nversion 1.0')

In [12]:
from typing import List
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct

In [13]:
class Qdrant:
    def __init__(self, host = "10.100.140.54", port = 6333):
        self.client = QdrantClient(host= host, port= port)
    def create_collection(self, collection_name):
        existing_collections = [col.name for col in self.client.get_collections().collections]
        if collection_name not in existing_collections:
            self.client.create_collection(
                collection_name = collection_name,
                vectors_config = VectorParams(
                    size=len(embeddings.embed_query("wellcome to DXTech")), distance=Distance.COSINE
                )
            )
    def insert_documents(self, collection_name: str, doc_chunks: List, embeddings):
        vectors = [
            embeddings.embed_query(chunk.page_content) for chunk in doc_chunks
        ]
        payloads = [
            {"id":i, "text": chunk.page_content} for i, chunk in enumerate(doc_chunks)
        ]
        
        points = [
            PointStruct(id=i, vector=vectors[i]) for i in range(len(vectors))
        ]
        self.client.upload_collection(collection_name=collection_name, vectors=vectors, payload=payloads)

    def search_database(self, collection_name, question, embeddings, limit=3):
        query_vector = embeddings.embed_query(question)
        search_result = self.client.query_points(
            collection_name=collection_name,
            query=query_vector,
            limit=limit
        ).points
        if not search_result:
            return "Không tìm thấy kết quả."
        try:
            text_result = "\n\n".join(hit.payload['text'] for hit in search_result)
            return text_result
        except KeyError as e:
            return "Lỗi: Cấu trúc payload không hợp lệ."
        except Exception as e:
            return "Lỗi: Không thể xử lý kết quả tìm kiếm."

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable.passthrough import RunnablePassthrough
from langchain_core.output_parsers.string import StrOutputParser

In [54]:
prompt_template = """
Bạn là một trợ lý AI thông minh. Tên bạn là DetNam. Bạn có quyền truy cập vào tài liệu/tri thức được cung cấp từ nguồn bên ngoài. Dựa trên thông tin đó, hãy trả lời theo cách chính xác, tự nhiên và hữu ích nhất.
Conversation history:
{history}

Trả lời câu hỏi: {question}  
- Dựa trên thông tin từ nguồn dữ liệu, tổng hợp và trình bày câu trả lời. Nếu không đủ thông tin, báo rõ và bổ sung kiến thức chung nếu phù hợp.
- Nếu câu hỏi không nằm trong tài liệu và không liên quan tới tài liệu, hãy sử dụng kiến thức bên ngoài để trả lời.
Retrieved context:
{context}

[Details]  
- Sử dụng ngôn ngữ tự nhiên, ngắn gọn.  
- Định dạng: [Tóm tắt] - [Giải thích]- [Ví dụ mẫu]- [Flow chart] - [Kết luận].
- Đưa thêm gợi ý câu hỏi tiếp theo

[Example]
- Xin chào/hi/hello -> tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây
"""

In [55]:
def build_chain(llm, qdrant_client, collection_name, embeddings, question, history):
    retrieved_context = qdrant_client.search_database(collection_name, question, embeddings)
    
    pipeline = (
        RunnablePassthrough()
        | {"history": lambda x: history, "question": lambda x: question, "context": lambda x: retrieved_context}
        | ChatPromptTemplate.from_template(prompt_template)
        | llm
        | StrOutputParser()
    )
    result = pipeline.invoke({})
    return result

In [ ]:
collection_name = "20250225"
qdrant_client = Qdrant()
qdrant_client.create_collection(collection_name)
qdrant_client.insert_documents(collection_name=collection_name, doc_chunks=docs, embeddings=embeddings)
question = input("Hãy nhập câu hỏi vào đây nhé: ")
historys = []
history_string = "\n".join(f"User: {chat['user']}\nBot: {chat['bot']}" for chat in historys)
while True:
    try:
        # Kiểm tra nếu người dùng muốn thoát
        if question.lower() in ["cancel", "thoát", "exit"]:
            print("Đã thoát chương trình.")
            break
        
        res = build_chain(llm, qdrant_client, collection_name=collection_name, embeddings=embeddings, question=question, history = history_string)
        print("---" * 50)
        print(f"Câu trả lời: {res}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")
        historys.append({
            "user": question,
            "bot": res
        })
    except KeyboardInterrupt:
        print("\nĐã thoát chương trình bằng Ctrl+C.")
        break
    except Exception as e:
        print(f"Lỗi: {e}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")

Hãy nhập câu hỏi vào đây nhé:  xin chào


------------------------------------------------------------------------------------------------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Tôi có thể giúp gì cho bạn hôm nay?

Bạn có cần hỗ trợ về một chủ đề cụ thể hay chỉ muốn trò chuyện? Hãy cho tôi biết, tôi sẽ cố gắng trả lời một cách chính xác và hữu ích nhất có thể.

Nếu bạn cần giúp đỡ về một vấn đề cụ thể, chẳng hạn như tìm kiếm thông tin, giải quyết vấn đề hoặc tạo báo cáo, hãy cho tôi biết chủ đề bạn quan tâm. Tôi sẽ cố gắng cung cấp thông tin chi tiết và hữu ích.

Hãy bắt đầu cuộc trò chuyện của chúng ta!


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  tôi đang muốn biết về hệ thống eVision


------------------------------------------------------------------------------------------------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Bạn muốn biết về hệ thống eVision?

[Tóm tắt]
Hệ thống eVision là một phần của hệ thống giám sát hành vi (EVision) được sử dụng để theo dõi và phân tích các hoạt động trên mạng.

[Giải thích]
Hệ thống eVision sử dụng công nghệ máy học và trí tuệ nhân tạo để phân tích các dữ liệu từ các nguồn khác nhau, bao gồm cả dữ liệu từ camera, microphone, và các thiết bị khác. Mục tiêu của hệ thống là giúp cho người dùng có thể theo dõi và phân tích các hoạt động trên mạng một cách hiệu quả hơn.

[Ví dụ mẫu]
Ví dụ, nếu bạn đang sử dụng hệ thống eVision để giám sát một khu vực, nó sẽ có thể nhận được tín hiệu từ camera và microphone, sau đó phân tích và cung cấp cho bạn thông tin về các hoạt động như người đi qua khu vực, tiếng ồn, hoặc các sự kiện khác.

[Flow chart]
```
            +---------

In [41]:
from langchain.tools import tool
from duckduckgo_search import DDGS

@tool
def duckduckgo_search(query: str) -> str:
    """Tìm kiếm thông tin trên DuckDuckGo dựa trên câu hỏi."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=3)]
        return "\n".join(results) if results else "Không tìm thấy kết quả trên DuckDuckGo."

In [42]:
duckduckgo_search("giá bitcoin hiện tại có giá bao nhiêu ?")

'Xem giá hiện tại, khối lượng giao dịch, lịch sử biểu đồ, v.v. Mua Crypto. Thị trường. Giao dịch. Cơ bản. ... Bitcoin hiện có giá trị bao nhiêu khi tính theo các loại tiền tệ khác? BTC đến USD. $96,334.37. BTC đến TRY ₺3,510,424.40. BTC đến RUB ₽8,557,381.97.\nGiá trị của Bitcoin là bao nhiêu? Bitcoin là đồng tiền mã hóa có giá trị nhất trên thế giới hiện nay. BTC được tạo ra vào năm 2009 bởi một hoặc một nhóm lập trình viên ẩn danh với bút danh Satoshi Nakamoto.\nBitcoin hôm nay có giá trị bao nhiêu? Hiện tại, một Bitcoin trị giá $63.547,10. Bạn đã tìm đến đúng nơi để tìm hiểu các câu trả lời và thông tin chuyên sâu về chuyển động giá của Bitcoin. Khám phá các biểu đồ Bitcoin mới nhất và giao dịch có trách nhiệm với OKX.'

In [43]:
prompt_template = """
Bạn là một trợ lý AI thông minh. Tên bạn là DetNam. Bạn có quyền truy cập vào tài liệu/tri thức được cung cấp từ nguồn bên ngoài. Dựa trên thông tin đó, hãy trả lời theo cách chính xác, tự nhiên và hữu ích nhất.
Conversation history:
{history}

Trả lời câu hỏi: {question}  
- Dựa trên thông tin từ nguồn dữ liệu, tổng hợp và trình bày câu trả lời. Nếu không đủ thông tin, báo rõ và bổ sung kiến thức chung nếu phù hợp.
- Nếu câu hỏi không nằm trong tài liệu và không liên quan tới tài liệu, hãy sử dụng kiến thức bên ngoài để trả lời.
Retrieved context:
{context}

[Details]  
- Sử dụng ngôn ngữ tự nhiên, ngắn gọn.  
- Định dạng: [Tóm tắt] - [Giải thích] - [Ví dụ mẫu] - [Flow chart] - [Kết luận].
- Đưa thêm gợi ý câu hỏi tiếp theo

[Hướng dẫn]  
- **Ưu tiên thông tin từ tài liệu**: Luôn dùng dữ liệu trong `context` trước. Nếu `context` không đủ hoặc không liên quan, cân nhắc dùng tools khi được phép.  
- **Sử dụng tools**: Chỉ gọi tools (duckduckgo_search) khi người dùng yêu cầu "Sử dụng nguồn tài liệu bên ngoài" hoặc khi `context` không cung cấp câu trả lời đầy đủ và cần bổ sung. Ghi rõ nguồn khi dùng tool (e.g., "Dựa trên DuckDuckGo search").  
- **Trường hợp đặc biệt**: Nếu câu hỏi là lời chào ("xin chào", "hi", "hello", "tên là gì"), trả lời theo mẫu trong [Example] mà không cần `context` hay tools.
- Sau khi dùng tool một lần, dừng tìm kiếm và trả lời ngay.

[Tools Available]  
- `duckduckgo_search`: Tìm kiếm thông tin trên DuckDuckGo, một công cụ tìm kiếm bảo vệ quyền riêng tư.  

[Example]
- Xin chào/hi/hello -> tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây
- Bạn tên là gì -> Tôi tên DetNam, sẽ hỗ trợ bạn mọi câu hỏi, và kể cả các thông tin mới nhất hôm nay
"""

In [44]:
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor

In [45]:
def build_chain(llm, qdrant_client, collection_name, embeddings, question, history="", tools=None):
    retrieved_context = qdrant_client.search_database(collection_name, question, embeddings)
    
    prompt = PromptTemplate(input_variables=["history", "question", "context"], template=prompt_template)
    
    # Nếu không có tools, dùng LLMChain
    if not tools:
        chain = LLMChain(llm=llm, prompt=prompt)
        result = chain.invoke({"history": history, "question": question, "context": retrieved_context})
        return result["text"]
    
    # Nếu có tools, dùng agent
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )

    # executor = AgentExecutor(
    #     agent=agent,
    #     tools=tools,
    #     max_iterations=2,  # Giới hạn 2 lần lặp
    #     verbose=True
    # )
    
    # Truyền input đúng định dạng cho agent
    result = agent.invoke({"input": question, "history": history, "context": retrieved_context})
    return result["output"]

In [46]:
tools = [duckduckgo_search]

In [47]:
qdrant_client = Qdrant()
qdrant_client.create_collection("20250225")
qdrant_client.insert_documents(collection_name="20250225", doc_chunks=docs, embeddings=embeddings)

conversation_history = ""
question = input("Hãy nhập câu hỏi vào đây nhé: ")

while True:
    try:
        if question.lower() in ["cancel", "thoát", "exit"]:
            print("Đã thoát chương trình.")
            break
        
        if question.lower() in ["xin chào", "hi", "hello"]:
            res = "Tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây?"
        else:
            use_external = "Sử dụng nguồn tài liệu bên ngoài" in question
            if use_external:
                question_cleaned = question.replace("Sử dụng nguồn tài liệu bên ngoài", "").strip()
                res = build_chain(llm, qdrant_client, "20250225", embeddings, question_cleaned, conversation_history, tools=tools)
            else:
                res = build_chain(llm, qdrant_client, "20250225", embeddings, question, conversation_history)
        
        print("---" * 20)
        print(f"Câu trả lời: {res}")
        conversation_history += f"User: {question}\nDetNam: {res}\n\n"
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")
    
    except KeyboardInterrupt:
        print("\nĐã thoát chương trình bằng Ctrl+C.")
        break
    except Exception as e:
        print(f"Lỗi: {e}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")

Hãy nhập câu hỏi vào đây nhé:  xin chào


------------------------------------------------------------
Câu trả lời: Tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây?


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  tôi muốn api login


--------------------------------------------------
------------------------------------------------------------
Câu trả lời: [Tóm tắt] - Câu hỏi: API login  
[Giải thích] - eVision cung cấp API để thực hiện các chức năng chính như quản lý người dùng, thông tin đăng ký, hồ sơ, xác minh liên quan, thống kê lưu lượng và kết nối camera IP. Tuy nhiên, không có thông tin cụ thể về cách thực hiện API login.

[Ví dụ mẫu] - Để sử dụng API login của eVision, bạn cần cung cấp thông tin đăng nhập của người dùng và thực hiện yêu cầu API theo hướng dẫn của hệ thống.

[Flow chart] - Không có thông tin cụ thể về flow chart cho API login của eVision.

[Kết luận] - Để biết thêm thông tin về cách thực hiện API login của eVision, bạn cần tham khảo tài liệu chính thức hoặc liên hệ với bộ phận hỗ trợ của hệ thống.

Gợi ý câu hỏi tiếp theo: Bạn có thể cung cấp thông tin về cách sử dụng API để kết nối camera IP với hệ thống AI không?


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  có bao nhiêu thành phần trong hệ thống evision


...

--------------------------------------------------


> Entering new AgentExecutor chain...
--------------------------------------------------
Thought: Tôi cần tìm kiếm thông tin về giá vàng hôm nay trên DuckDuckGo.

Action: duckduckgo_search
Action Input: "giá vàng hôm nay"

Observation: Giá vàng hôm nay, hôm qua trong nước và thế giới mới nhất, Biến động giá vàng Sjc, 9999, 24k, 18k của PNJ, Doji trong ngày. Phân Tích, dự báo xu hướng giá vàng trong nước
Cập Nhật Mới Nhất Bảng Giá Vàng Hôm Nay: PNJ, SJC, Vàng 9999, 18K, 24K ️Giá Miếng Vàng ⭐️Trong Nước ️Giá Vàng Thế Giới ☎️1800545457 (Miễn Phí)
Xem giá vàng ngày hôm nay 24h mới nhất: giá vàng SJC, giá vàng 9999, giá vàng PNJ, DOJI, 24k, 18k... cập nhật từng giờ
Thought:--------------------------------------------------
Question: Giá vàng hôm nay
Thought: Tôi cần tìm kiếm thông tin về giá vàng hôm nay trên DuckDuckGo.

Action: duckduckgo_search
Action Input: "giá vàng hôm nay"

Observation: Giá vàng hôm nay, hôm qua trong nước và thế g

Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  Sử dụng nguồn tài liệu bên ngoài: giá vàng có mức giá trong hôm nay


--------------------------------------------------


> Entering new AgentExecutor chain...
--------------------------------------------------
Thought: Tôi cần tìm kiếm thông tin về giá vàng hiện tại trên DuckDuckGo.

Action: duckduckgo_search
Action Input: "giá vàng hôm nay"

Observation: Cập Nhật Mới Nhất Bảng Giá Vàng Hôm Nay: PNJ, SJC, Vàng 9999, 18K, 24K ️Giá Miếng Vàng ⭐️Trong Nước ️Giá Vàng Thế Giới ☎️1800545457 (Miễn Phí)
Giá vàng hôm nay, hôm qua trong nước và thế giới mới nhất, Biến động giá vàng Sjc, 9999, 24k, 18k của PNJ, Doji trong ngày. Phân Tích, dự báo xu hướng giá vàng trong nước
Giá vàng hôm nay 24-2: Vừa mở cửa đã đồng loạt giảm. Giá vàng hôm nay trên thị trường quốc tế giảm ngay khi vừa mở cửa tuần giao dịch mới, kéo giá vàng miếng SJC, vàng nhẫn cùng đi xuống. Ngày 24-2, giá vàng thế giới mở cửa tuần giao dịch lùi sâu về 2.924 USD/ounce ...
Thought:--------------------------------------------------
Question: Giá vàng có mức giá trong hôm nay
Thought: Tôi cần tìm ki

Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  exit


Đã thoát chương trình.


In [48]:
prompt_template = """
    Bạn là trợ lý AI thông minh và rất nhiều kiến thức phong phú, tên riêng của bạn là DetNam. Chuyên cung cấp câu trả lời chính xác và hữu ích bằng cách kết hợp dữ liệu nội bộ (private date) và dữ 
    liệu bên ngoài (public data).
    [Conversation history]: {history}
    [Trả lời câu hỏi]: {question}  
    - Dựa trên thông tin từ nguồn dữ liệu, tổng hợp và trình bày câu trả lời. Nếu không đủ thông tin, báo rõ và bổ sung kiến thức chung nếu phù hợp.
    - Nếu câu hỏi không nằm trong tài liệu và không liên quan tới tài liệu, hãy sử dụng kiến thức bên ngoài để trả lời.
    [Retrieved context]: {context}
    [Hướng dẫn]  
    - **Ưu tiên thông tin từ tài liệu**: Luôn dùng dữ liệu trong `context` trước. Nếu `context` không đủ hoặc không liên quan, cân nhắc dùng tools khi được phép.  
    - **Sử dụng tools**: Chỉ gọi tools (duckduckgo_search) khi bắt hiện câu hỏi sử dụng các câu từ liên quan tới thời gian thực (hôm nay, hôm qua, tháng này, ...) hoặc khi `context` không cung cấp câu trả lời đầy đủ và cần bổ sung. Ghi rõ nguồn khi dùng tool (e.g., "Dựa trên DuckDuckGo search").  
    - **Trường hợp đặc biệt**: Nếu câu hỏi là lời chào ("xin chào", "hi", "hello", "tên là gì"), trả lời theo mẫu trong [Example] mà không cần `context` hay tools.
    - Sau khi dùng tool một lần, dừng tìm kiếm và trả lời ngay.
    [Details]
    - Sử dụng ngôn ngữ tự nhiên, ngắn gọn.
    - Hạn chế tự bịa các thông tin, các thông tin phải có trích dẫn.
    [Example]
    - Xin chào/hi/hello -> tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây
    - Bạn tên là gì -> Tôi tên DetNam, sẽ hỗ trợ bạn mọi câu hỏi, và kể cả các thông tin mới nhất hôm nay
    
"""

In [53]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from duckduckgo_search import DDGS

# Tool DuckDuckGo Search
@tool
def duckduckgo_search(query: str) -> str:
    """Tìm kiếm thông tin trên DuckDuckGo."""
    with DDGS() as ddgs:
        results = [r['body'] for r in ddgs.text(query, max_results=3)]
        return "\n".join(results) if results else "Không tìm thấy kết quả."

tools = [duckduckgo_search]

def build_chain(llm, qdrant_client, collection_name, embeddings, question, history="", tools=None):
    retrieved_context = qdrant_client.search_database(collection_name, question, embeddings)
    
    prompt = PromptTemplate(input_variables=["history", "question", "context"], template=prompt_template)
    
    # Nếu không có tools, dùng LLMChain
    if not tools:
        chain = LLMChain(llm=llm, prompt=prompt)
        result = chain.invoke({"history": history, "question": question, "context": retrieved_context})
        return result["text"]
    
    # Nếu có tools, dùng agent
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=True
    )

    # executor = AgentExecutor(
    #     agent=agent,
    #     tools=tools,
    #     max_iterations=2,  # Giới hạn 2 lần lặp
    #     verbose=True
    # )
    
    # Truyền input đúng định dạng cho agent
    result = agent.invoke({"input": question, "history": history, "context": retrieved_context})
    return result["output"]

# Vòng lặp chính
qdrant_client = Qdrant()
qdrant_client.create_collection("20250225")
qdrant_client.insert_documents(collection_name="20250225", doc_chunks=docs, embeddings=embeddings)

conversation_history = ""
question = input("Hãy nhập câu hỏi vào đây nhé: ")

while True:
    try:
        if question.lower() in ["cancel", "thoát", "exit"]:
            print("Đã thoát chương trình.")
            break
        
        if question.lower() in ["xin chào", "hi", "hello"]:
            res = "Tôi tên DetNam, trợ lý thông minh, tôi có thể giúp gì cho bạn đây?"
        else:
            res = build_chain(llm, qdrant_client, "20250225", embeddings, question, conversation_history)
        
        print("---" * 20)
        print(f"Câu trả lời: {res}")
        conversation_history += f"User: {question}\nDetNam: {res}\n\n"
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")
    
    except KeyboardInterrupt:
        print("\nĐã thoát chương trình bằng Ctrl+C.")
        break
    except Exception as e:
        print(f"Lỗi: {e}")
        question = input("Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C): ")

Hãy nhập câu hỏi vào đây nhé:  tôi muốn biết về evision


--------------------------------------------------
------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Tôi có thể giúp gì cho bạn?

Về câu hỏi "eVision", tôi không tìm thấy thông tin cụ thể trong tài liệu phân tích nghiệp vụ hệ thống giám sát hành vi EVISION version 1.0. Tuy nhiên, tôi có thể cung cấp một số thông tin chung về chủ đề này.

EVision là một hệ thống giám sát hành vi được sử dụng để theo dõi và phân tích các hoạt động trên mạng internet. Hệ thống này được thiết kế để giúp cơ quan chức năng và tổ chức bảo mật phát hiện và ngăn chặn các hoạt động bất hợp pháp như gian lận, buôn bán ma túy, và các hoạt động khác.

Tuy nhiên, tôi không có thông tin cụ thể về hệ thống EVISION, bao gồm cả phiên bản 1.0. Nếu bạn có thêm thông tin hoặc câu hỏi cụ thể về chủ đề này, tôi sẽ cố gắng giúp đỡ.


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  bạn bịa thông tin à


--------------------------------------------------
------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Tôi có thể giúp gì cho bạn?

Về câu hỏi "eVision", tôi không tìm thấy thông tin cụ thể trong tài liệu phân tích nghiệp vụ hệ thống giám sát hành vi EVISION version 1.0. Tuy nhiên, tôi có thể cung cấp một số thông tin chung về chủ đề này.

EVision là một hệ thống giám sát hành vi được sử dụng để theo dõi và phân tích các hoạt động trên mạng internet. Hệ thống này được thiết kế để giúp cơ quan chức năng và tổ chức bảo mật phát hiện và ngăn chặn các hoạt động bất hợp pháp như gian lận, buôn bán ma túy, và các hoạt động khác.

Tuy nhiên, tôi không có thông tin cụ thể về hệ thống EVISION, bao gồm cả phiên bản 1.0. Nếu bạn có thêm thông tin hoặc câu hỏi cụ thể về chủ đề này, tôi sẽ cố gắng giúp đỡ.


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  trả lời sai rồi


--------------------------------------------------
------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Tôi có thể giúp gì cho bạn?

Về câu hỏi "eVision", tôi không tìm thấy thông tin cụ thể trong tài liệu phân tích nghiệp vụ hệ thống giám sát hành vi EVISION version 1.0. Tuy nhiên, tôi có thể cung cấp một số thông tin chung về chủ đề này.

EVision là một hệ thống giám sát hành vi được sử dụng để theo dõi và phân tích các hoạt động trên mạng internet. Hệ thống này được thiết kế để giúp cơ quan chức năng và tổ chức bảo mật phát hiện và ngăn chặn các hoạt động bất hợp pháp như gian lận, buôn bán ma túy, và các hoạt động khác.

Tuy nhiên, tôi không có thông tin cụ thể về hệ thống EVISION, bao gồm cả phiên bản 1.0. Nếu bạn có thêm thông tin hoặc câu hỏi cụ thể về chủ đề này, tôi sẽ cố gắng giúp đỡ.

Nếu bạn có thêm thông tin hoặc câu hỏi cụ thể về chủ đề này, tôi sẽ cố gắng giúp đỡ.


Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  giá vàng hôm nay


--------------------------------------------------
------------------------------------------------------------
Câu trả lời: Xin chào! Tôi là DetNam, trợ lý thông minh. Tôi có thể giúp gì cho bạn?

Về câu hỏi "giá vàng hôm nay", tôi không tìm thấy thông tin cụ thể trong tài liệu phân tích nghiệp vụ hệ thống giám sát hành vi EVISION version 1.0. Tuy nhiên, tôi có thể cung cấp một số thông tin chung về giá vàng.

Giá vàng là một loại hàng hóa quan trọng và được theo dõi chặt chẽ trên thị trường. Giá vàng thường được xác định bởi các yếu tố như nhu cầu, cung, và các sự kiện kinh tế - chính trị.

Dựa trên thông tin từ nguồn dữ liệu, giá vàng hôm nay có thể được tìm thấy trên các trang web tài chính hoặc các ứng dụng phân tích thị trường. Tuy nhiên, tôi không có thông tin cụ thể về giá vàng hôm nay tại thời điểm này.

Nếu bạn muốn biết giá vàng hôm nay, tôi khuyên bạn nên kiểm tra các trang web tài chính hoặc các ứng dụng phân tích thị trường để có thông tin cập nhật và chính xác nhất.

Xin

Hãy nhập câu hỏi tiếp theo (hoặc nhập 'cancel' để thoát, hoặc nhấn Ctrl+C):  exit
